In [5]:
import numpy as np
import pandas as pd
from scipy.stats import qmc 
from scipy.interpolate import interp1d 
from tqdm import tqdm
import time

import sys, os, pathlib

NOTEBOOK_DIR = pathlib.Path(os.getcwd())  
ROOT = NOTEBOOK_DIR.parents[0]  
sys.path.insert(0, str(ROOT))
from src.Heston_FFT import heston_price_fft


n_samples = 25_000

# Market-level constants
S0 = 105.0
q  = 0.0

alpha = 1.5
n = 4096
eta = 0.1
lam = 2.0*np.pi / (n * eta)  


def sample_heston_lhs(
    n_samples: int,
    seed: int = 42,
    moneyness_bounds=(0.6, 1.4),
    T_bounds=(0.09, 1.40),
    r_bounds=(-0.01, 0.10)
) -> pd.DataFrame:
    """
    Generate n_samples plausible Heston parameters via Latin Hypercube Sampling (LHS),
    and also sample option strikes through moneyness m = K / S0.

    Constraints (as implemented by bounds below):
      - v0     in [0.01, 0.09]
      - kappa  in [0.50, 5.00]
      - theta  in [0.01, 0.09]
      - sigma_v in [0.10, 1.00]
      - rho    in [-0.95, -0.10]
      - T      in [0.09, 1.40]
      - r      in [-0.01, 0.10]
      - Feller: 2*kappa*theta >= sigma_v^2
      - moneyness m in [m_low, m_high]

    Returns
    -------
    pd.DataFrame
        Columns: ['S0','K','m','T','r','q','v0','kappa','theta','sigma_v','rho'].
    """
    # bounds
    m_low, m_high = moneyness_bounds
    if not (m_low > 0 and m_high > m_low):
        raise ValueError("moneyness_bounds must satisfy 0 < m_low < m_high.")
    T_low, T_high = T_bounds
    r_low, r_high = r_bounds

    # order: v0, kappa, theta, sigma_v, rho, m, T, r
    l_bounds = np.array([0.01, 0.5, 0.01, 0.10, -0.95, m_low, T_low, r_low], dtype=float)
    u_bounds = np.array([0.09, 5.0, 0.09, 1.00, -0.10, m_high, T_high, r_high], dtype=float)

    collected = []
    attempt = 0

    # Repeat in batches until we have enough valid samples
    while len(collected) < n_samples and attempt < 20:
        # Oversample to compensate for filtering
        batch_size = int(np.ceil((n_samples - len(collected)) * 2.0))
        sampler = qmc.LatinHypercube(d=8, seed=seed + attempt)
        U = sampler.random(batch_size)
        X = qmc.scale(U, l_bounds, u_bounds)  # shape [batch_size, 8]

        v0, kappa, theta, sigma_v, rho, m, T_samp, r_samp = (
            X[:, 0], X[:, 1], X[:, 2], X[:, 3], X[:, 4], X[:, 5], X[:, 6], X[:, 7]
        )

        # Feller condition to ensure variance stays strictly positive
        feller_ok = (2.0 * kappa * theta) >= (sigma_v ** 2)

        X_ok = X[feller_ok]

        # Append valid rows until reaching n_samples
        for row in X_ok:
            collected.append(row)
            if len(collected) >= n_samples:
                break

        attempt += 1

    if len(collected) < n_samples:
        raise RuntimeError(
            "Could not generate enough samples satisfying constraints. "
            "Relax bounds slightly or increase attempts."
        )

    params = np.vstack(collected)[:n_samples]
    # columns must match the order used above
    df = pd.DataFrame(params, columns=["v0", "kappa", "theta", "sigma_v", "rho", "m", "T", "r"])

    # Compute K from moneyness and add market columns
    df["S0"] = S0
    df["K"]  = (df["m"] * S0).astype(float)
    df["q"]  = q
   
    # Reorder columns (KEEP S0, K, q for pricing)
    df = df[["S0","K","m","T","r","q","v0","kappa","theta","sigma_v","rho"]]

    return df

df_first = sample_heston_lhs(n_samples, seed=123)
df_first.head()


,S0,K,m,T,r,q,v0,kappa,theta,sigma_v,rho
0,105.0,146.642476,1.396595,0.740600,0.024181,0.0,0.035041,4.238505,0.084534,0.518515,-0.788792
1,105.0,70.197749,0.668550,0.329994,-0.008213,0.0,0.056062,0.972229,0.059807,0.157281,-0.437313
2,105.0,119.304675,1.136235,0.146670,0.064912,0.0,0.028813,3.440158,0.082486,0.257396,-0.447415
3,105.0,84.439429,0.804185,0.618756,0.035695,0.0,0.034451,1.921728,0.057633,0.274679,-0.365590
4,105.0,74.421070,0.708772,0.414266,0.076979,0.0,0.035230,1.219318,0.068533,0.354979,-0.605301


In [ ]:
def price_heston_fft_df(
    df: pd.DataFrame,
    option_type: str = "call",
    alpha: float = 1.5,
    N: int = 4096,
    eta: float = 0.05,
) -> pd.DataFrame:
    """
    Aggiunge la colonna 'price' con il prezzo Heston per ciascuna riga di df,
    utilizzando la funzione heston_price_fft già importata.

    Richiede che df contenga le colonne:
    ['S0','K','T','r','q','v0','kappa','theta','sigma_v','rho'].
    """
    required = ["S0","K","T","r","q","v0","kappa","theta","sigma_v","rho"]
    missing = [c for c in required if c not in df.columns]
    if missing:
        raise ValueError(f"Mancano colonne richieste nel DataFrame: {missing}")

    prices = []
    for _, row in df.iterrows():
        try:
            p = heston_price_fft(
                S=float(row["S0"]),
                K=float(row["K"]),
                T=float(row["T"]),
                r=float(row["r"]),
                v0=float(row["v0"]),
                kappa=float(row["kappa"]),
                theta=float(row["theta"]),
                sigma_v=float(row["sigma_v"]),
                rho=float(row["rho"]),
                option_type=option_type,
                q=float(row["q"]),
                alpha=alpha,
                N=N,
                eta=eta,
            )
        except Exception:
            # Se qualcosa va storto per una riga (es. instabilità numerica), metti NaN
            p = np.nan
        prices.append(p)

    out = df.copy()
    out["price"] = prices
    return out


df = price_heston_fft_df(df_first)
df.head()

,S0,K,m,T,r,q,v0,kappa,theta,sigma_v,rho,price
0,105.0,146.642476,1.396595,0.740600,0.024181,0.0,0.035041,4.238505,0.084534,0.518515,-0.788792,0.306700
1,105.0,70.197749,0.668550,0.329994,-0.008213,0.0,0.056062,0.972229,0.059807,0.157281,-0.437313,34.625894
2,105.0,119.304675,1.136235,0.146670,0.064912,0.0,0.028813,3.440158,0.082486,0.257396,-0.447415,0.179122
3,105.0,84.439429,0.804185,0.618756,0.035695,0.0,0.034451,1.921728,0.057633,0.274679,-0.365590,23.101824
4,105.0,74.421070,0.708772,0.414266,0.076979,0.0,0.035230,1.219318,0.068533,0.354979,-0.605301,32.995791


In [7]:
# check
price_fft_example = heston_price_fft(S=105, K=74.4210, T=0.4142, r=0.0770, v0=0.0352, kappa=1.2193, theta=0.0685, sigma_v=0.3550, rho=-0.6053,
                             option_type="call", q=0.0, alpha=alpha, N=n, eta=eta)

price_fft_example

32.999084677497365

In [8]:
df.drop(columns=['q', 'S0', 'K'], inplace = True)
df

,m,T,r,v0,kappa,theta,sigma_v,rho,price
0,1.396595,0.740600,0.024181,0.035041,4.238505,0.084534,0.518515,-0.788792,0.306700
1,0.668550,0.329994,-0.008213,0.056062,0.972229,0.059807,0.157281,-0.437313,34.625894
2,1.136235,0.146670,0.064912,0.028813,3.440158,0.082486,0.257396,-0.447415,0.179122
3,0.804185,0.618756,0.035695,0.034451,1.921728,0.057633,0.274679,-0.365590,23.101824
4,0.708772,0.414266,0.076979,0.035230,1.219318,0.068533,0.354979,-0.605301,32.995791
...,...,...,...,...,...,...,...,...,...
24995,1.153616,0.259050,0.013816,0.044753,4.471891,0.037734,0.117173,-0.365748,0.462908
24996,1.339221,1.118867,0.054329,0.025421,4.704041,0.012900,0.329857,-0.438162,0.117516
24997,0.849864,0.406609,0.003529,0.029538,2.687529,0.066707,0.520770,-0.405233,16.854891
24998,0.682701,0.116009,0.076821,0.016816,4.468869,0.061329,0.234115,-0.329437,33.944210


In [ ]:
import os
os.makedirs("data", exist_ok=True)
#df.to_csv("data/df_prova.csv", index=False)